# 运算符重载
运算符重载指自定义对象使用的中缀运算符或已于运算符，在Python中函数调用（()）,属性访问（.），以及元素访问/切片（[]）也是运算符。
# 重载规则
1.不能重载内置类型的运算符  
2.不能新建运算符  
3.有些运算符不可重载，如is、and、or、not  
# 一元运算符
-（__neg__）,+（__pos__）、~（__invert__）。重载运算符的一个原则是始终返回一个新对象（除了增量赋值等运算符），不能修改self。
# 重载加法运算符
对于序列来说，+运算符应该用于拼接，*运算符用于重复复制。但对于之前实现的Vector序列类型，+和*应当用于向量运算。
# 左加法运算符
对于a + b而言，python对特殊方法的调用如下：  
1.若a有\__add__且不返回NotImplemented则调用，否则看b有没有\__radd__  
2.若b有\__radd__且不返回NotImplemented则调用该反向方法，否则失败抛出TypeError

In [1]:
# 如第十章的Vector（序列），可以重载+运算符用于向量运算
def __add__(self, other):
    # 必须捕获TypeError异常，一旦出错，可以抛出NotImplemented令python执行备选方案以支持其他类型。
    try:
        # 鸭子类型，被加者必须是有效的可迭代对象（字符串可迭代，但无法相加）
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        # 返回新对象
        return Vector(a+b for a, b in pairs)
    except TypeError:
        return NotImplemented

def __radd__(self, other):
    # 委托给__add__
    return self + other

# 重载乘法运算符
重载标量乘法，对于向量而言，标量乘法需要另外一个操作数是实数，在这种场景下不像上面对于向量加法采用的‘鸭子类型’，因为相比之下标量乘法对操作数的要求更加具体，因此这里更合理的做法是检查其是否是numbers.Real（抽象基类）的子类实例，也就是采用‘白鹅类型’。

In [1]:
def __mul__(self, scalar):
    # 白鹅类型
    if isinstance(scalar, numbers.Real):
        return Vector(n * scalar for n in self)
    else:
        return NotImplemented
    
def __rmul__(self, scalar):
    # 委托__mul__
    return self * scalar


# 比较运算符
比较运算符的调用规则和上面类似，不过对于==，反向调用的仍然是对方的\__eq__，对>则反向调用对方的<（__less__）。对于==和!=，如果反向调用还失败（第二个操作数没实现特殊方法或返回NotImplemented）则会比较对象的id。


一般而言object默认的!=（\__ne__）即可满足需求，它相当于对==的结果去取反。
# 相等运算符
之前的相等运算符（第十章）实现如下：

In [2]:
def __eq__(self, other):
    # 这里使用迭代比较的方式确定长度以及各个分量相等，而不是使用简单的构造tuple后交给元组判断相等，因为当分量较多时创建元组使得效率低下。
    return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))

之前的问题在于只要是可迭代的，长度相同的且分量相等的对象都是和自己等价的，这样的==实现太过宽泛因此要限定第二个操作数的类型。

In [4]:
def __eq__(self, other):
    if isinstance(other, Vector):
        # 这里使用迭代比较的方式确定长度以及各个分量相等，而不是使用简单的构造tuple后交给元组判断相等，因为当分量较多时创建元组使得效率低下。
        return (len(self) == len(other) and all(a == b for a, b in zip(self, other)))
    else:
        return NotImplemented

# 增量赋值运算符
如果增量赋值运算符的特殊方法没有实现，那么使用增量赋值运算时相当于语法糖，比如a+=b（不存在反向调用等后备机制），相当于a = a + b。因此如果只实现\__add__是可以使用+=的。

#### 在实现增量运算时，思想和对应的中缀运算如+不同，一般而言，+相当于要求第二个操作数和自身同类型（因此上面示例中向量加法可接受任意可迭代对象的做法是否合理，需要看具体使用场景），而+=则要宽容一些。比如内置的list，+要求第二个操作数是list对象，行为是返回新的拼接好的list，而+=则接受任何可迭代的操作数，行为和参数的宽容度相当于list.extend()，是一种就地修改操作。另外增量赋值特殊方法必须返回self。

另外，如果中缀运算符，如+，只处理与自身同类型的操作数，则没必要实现反向调用方法\__radd__，因为一般而言，__反向调用方法是为了处理第二个操作数类型不符而要求的备选方案（因此上面Vector捕获TypeError并返回NotImplemented来试图解决类型不兼容问题）__。